In [3]:
import json
with open("test_prompts.json", 'r', encoding='utf-8') as f:
    predictions = json.load(f)

print(predictions[str(1)])

{'messages': [{'role': 'system', 'content': "Given the following entity classes and sentences, label entity mentions with their respective classes in sentences according to the sentences' context. In the output, only include entity mentions and their respective class in the given output format. No needed further explanation.\nCONTEXT: entity classes: ['O', 'HolyBook', 'Messenger', 'Allah', 'Event', 'AstronomicalBody', 'Prophet', 'Person', 'Food', 'GeographicalLocation', 'Color', 'Angel', 'Artifact', 'Religion', 'AfterlifeLocation', 'Throne', 'Language'].\nExample sentence: Itu lebih baik daripada apa yang mereka kumpulkan.\n.\nExample output: Itu/O lebih/O baik/O daripada/O apa/O yang/O mereka/O kumpulkan/O ./O \n    "}, {'role': 'user', 'content': '\n    Test sentence: Tidak ada sesuatu yang lebih kecil dan yang lebih besar daripada itu, kecuali semua tercatat dalam kitab yang nyata (Lauh Mahfuz).\n\n    Test output:\n    '}]}


In [4]:
import time
import logging
from transformers import logging as hf_logging
import torch
import time
from transformers import BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
import torch
import re
import ast
import pandas as pd
import ast
from accelerate import Accelerator
from collections import deque
from rdflib import Graph
import json
from tqdm import tqdm


base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

device_map = {"": 0}
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(  ## If it fails at this line, restart the runtime and try again.
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True,
    low_cpu_mem_usage=True
)
model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
model.config.pretraining_tp = 1

def generate_answer(pipe, example):
    
    prompt = pipe.tokenizer.apply_chat_template(example["messages"][:2],
                                                tokenize=False,
                                                add_generation_prompt=True)
    terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
    
    outputs = pipe(prompt,
                max_new_tokens=1024,
                eos_token_id=terminators,
                do_sample=True,
                temperature=0.3,
                top_k=30,
                top_p=0.85,
                )
    generated_text = outputs[0]['generated_text']
    return {"content": example["messages"][1]['content'], "generated_text": generated_text}

def create_input_prompt(system_message, user_prompt):
    return {
        "messages": [
            {"role": "system","content": system_message},
            {"role": "user", "content": user_prompt},
        ]
    }

2024-09-17 13:52:56.380867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 13:52:56.393973: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 13:52:56.397983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 13:52:56.409092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 13:52:57.265242: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
predict_dict = dict()
num = 0
for testid in predictions:
    num +=1
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer) 
    # Generate answers for the dataset
    message = predictions[testid]
    results = generate_answer(pipe, message)
    outputs = results["generated_text"]
    select_outputs = outputs.split("\n\n")[-1]
    
    print("##############")
    print(num)
    print(select_outputs)
    print("###############")
    predict_dict[num] = select_outputs

##############
1
Tidak/O ada/O sesuatu/O yang/O lebih/O kecil/O dan/O yang/O besar/O daripada/O itu/O kecuali/O semua/O tercatat/O dalam/O kitab/O yang/O nyata/O (Lauh/O Mahfuz/O)
###############
##############
2
Ketahuilah/O bahwa/O sesungguhnya/O (/O bagi/O )/O para/O wali/O Allah/Allah/O itu/O tidak/O ada/O rasa/O takut/O (/O yang/O )/O menimpa/O mereka/O dan/O mereka/O pun/O tidak/O bersedih/O./O
###############
##############
3
Mereka/O adalah/O orang-orang/O yang/O beriman/O dan/O bertakwa/O./O
###############
##############
4
Bagi/O mereka/O berita/O gembira/O di/O dalam/O kehidupan/O di/O dunia/O dan/O di/O akhirat/O
###############
##############
5
Tidak/O ada/O perubahan/O bagi/O kalimat-kalimat/O (/O ketetapan/O dan/O janji/O )/O Allah/Allah./O
###############
##############
6
Demikian/O itulah/O kemenangan/O yang/O agung/O
###############
##############
7
Janganlah/O engkau/O (Prophet)/Nabi Muhammad/Person sedih/O oleh/O perkataan/O mereka/O./O
###############
#############

In [6]:
import json
# Save the dictionary to a JSON file
with open("results-zeroshot-attempt-1.json", "w") as json_file:
    json.dump(predict_dict, json_file, indent=4)  # 'indent' adds formatting for readability